# Save, Push and Pull

このチュートリアルでは、MINTOを使用して最適化実験を保存し、リモートリポジトリにプッシュする方法を説明します。

## Save

mintoではデータをローカルに保存する方法として、"ファイルベースの保存"と"OMMX Archiveベースの保存"の2つがあります。

### ファイルベースの保存

ファイルベースの保存については2つの保存方法があります。

``` python
experiment = minto.Experiment("sample", auto_saving=True)
```

上記のように`auto_saving=True`を指定すると、`.log_*`メソッドが呼び出されるたびにファイルベースの保存の形で自動的に保存されます。

また実験が終わった後に

``` python
experiment.save("path/to/directory")
```

のように`save`メソッドを呼び出すことで、指定したディレクトリにファイルベースの保存を行うこともできます。

### OMMX Archiveベースの保存

```python
experiment.save_as_ommx_archive("path/to/archive.minto.ommx")
```

のように`save_as_ommx_archive`メソッドを呼び出すことで、OMMX Archiveベースのひとつの圧縮ファイルとして保存することができます。



## GitHub PackageへのPushとPull

ここではOMMX Artifact機能を利用して、MINTOにより管理している実験をGitHub Packagesにpushする方法と、他のユーザがこれらの実験をpullする方法について説明します。  
OMMX Artifactは、実験結果を他のコンテナレジストリにpushすることも可能です。
GitHub Packagesにpushすることで、コードと実験結果の両方をGitHub上で管理でき量になり、チームでの共有も容易になります。

まずは数理最適化のための数値実験を行いましょう。
ここでは、["Quick Start"](/minto/quickstart)と同じものを題材にします。
詳細は元のチュートリアルページをご覧ください。

In [1]:
import minto
import ommx_pyscipopt_adapter as scip_ad
from ommx.dataset import miplib2017

instance_name = "reblock115"
instance = miplib2017(instance_name)

timelimit_list = [0.1, 0.5, 1, 2]

experiment = minto.Experiment("scip_reblock115", auto_saving=False, verbose_logging=False)

experiment.log_global_instance(instance_name, instance)
adapter = scip_ad.OMMXPySCIPOptAdapter(instance)
scip_model = adapter.solver_input

for timelimit in timelimit_list:
    with experiment.run() as run:
        run.log_parameter("timelimit", timelimit)

        # Solve by SCIP
        scip_model.setParam("limits/time", timelimit)
        scip_model.optimize()
        solution = adapter.decode(scip_model)

        run.log_solution("scip", solution)

experiment.get_run_table()

metadata        parameter solution_scip                      \
       elapsed_time run_id timelimit      feasible  name     objective   
run_id                                                                   
0          0.110150      0       0.1          True  scip  0.000000e+00   
1          0.407828      1       0.5          True  scip -2.824191e+07   
2          0.507013      2       1.0          True  scip -2.824191e+07   
3          1.007558      3       2.0          True  scip -2.824191e+07   

                                    
       optimality relaxation start  
run_id                              
0               0          0  None  
1               0          0  None  
2               0          0  None  
3               0          0  None

## GitHub　Packageへのpush

実験結果を、GitHub Packagesにpushする方法を学びましょう。

### 前提条件

1. GitHubのアカウントとリポジトリが必要となります。
2. まだ作成していない場合、リポジトリ作成に関するGitHubのドキュメントを参照してください。

GitHub Packagesにpushするには、GitHub ActionsでPersonal Access Token (PAT)もしくはGITHUB_TOKENを用いた認証が必要です。write:packages権限を持つPATを作成してください。

PATの作成方法については、[GitHubのドキュメント](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token)を参照してください。

取得したトークンを用いて以下の2つのいずれかの方法で認証を行います。


1. CLIを用いる方法
    ommx loginを実行し、認証情報をローカルマシンに保存します
    ```bash
    ommx login --username=<GITHUB_USERNAME> --password=<PERSONAL_ACCESS_TOKEN> https://ghcr.io
    ```
   
2. 環境変数を用いる方法
    - 以下の環境変数の設定を行います
   ```yaml
    env:
        OMMX_BASIC_AUTH_DOMAIN: ghcr.io
        OMMX_BASIC_AUTH_USERNAME: ${{ github.actor }}
        OMMX_BASIC_AUTH_PASSWORD: ${{ github.token }}
    ```

    - `OMMX_BASIC_AUTH_PASSWORD`には、GitHubのPersonal Access Tokenを作成する必要があります。Personal Access Tokenを作成するには、GitHubのガイドに従い、`write:packages`権限を有効にしてください
  
注意: GitHub Packagesへのpushとpullには、OMMX Artifact機能を用います。そのため、GitHub認証などの設定はOMMX Artifactと同じになります。詳細については、[こちらのノートブック](https://github.com/Jij-Inc/ommx/blob/main/notebooks/artifact.ipynb)を参照してください。

### push

認証の設定が完了したら、Experiment.push_githubメソッドを用いて、GitHub Packagesにpushします。
このメソッドは以下のパラメータを持ちます。

- org: GitHubでの組織名
- repo: GitHubリポジトリ名
- name: Artifact名 (初期設定では`Experiment.name`)
- tag: タグ名 (初期設定では`Experiment.timestamp.strftime('%Y%m%d%H%M%S')`)


In [37]:
# If environment variables are not set, please use the following to set them.
# Environment variables

# os.environ["OMMX_BASIC_AUTH_DOMAIN"] = 'ghcr.io'
# os.environ["OMMX_BASIC_AUTH_USERNAME"] = <GITHUB_USERNAME>
# os.environ["OMMX_BASIC_AUTH_PASSWORD"] = <PERSONAL_ACCESS_TOKEN>

早速、`Experiment.push_github`メソッドを用いて、GitHub Packagesにpushしてみましょう。
先述の通り、nameとtagのパラメータは指定する必要はありません。
このメソッドは`ommx.Artifact`オブジェクトを返し、`ommx.Artifact.image_name`を用いてpushされたイメージ名を確認することができます。

In [ ]:
artifact = experiment.push_github(org="Jij-Inc", repo="MINTO-Public")
artifact.image_name

'ghcr.io/jij-inc/minto-public/scip_reblock115:latest5'

## GitHub Packagesからのpull

次に、GitHub Packagesにpushしたイメージをpullしてみましょう。
そのために、`minto.Experiment.load_from_registry`メソッドを用います。
このメソッドはGitHub Packagesだけでなく、他のコンテナレジストリからのpullもサポートしています。
そのため、イメージ名を直接指定する必要があるのです。


In [ ]:
exp2 = minto.Experiment.load_from_registry(
    "ghcr.io/jij-inc/minto-public/scip_reblock115:latest5"
)

pullしてきたデータを見てみましょう。

In [42]:
exp2.get_run_table()

metadata        parameter solution_scip                      \
       elapsed_time run_id timelimit      feasible  name     objective   
run_id                                                                   
0          0.110150      0       0.1          True  scip  0.000000e+00   
1          0.407828      1       0.5          True  scip -2.824191e+07   
2          0.507013      2       1.0          True  scip -2.824191e+07   
3          1.007558      3       2.0          True  scip -2.824191e+07   

                                    
       optimality relaxation start  
run_id                              
0               0          0  None  
1               0          0  None  
2               0          0  None  
3               0          0  None

## まとめ

mintoを使ってデータをローカルに保存する方法（ファイルベース, OMMX Archiveベース）と GitHub Packagesへのpushとpullする方法を見てきました。

mintoはこのようにデータを取り扱いしやすくする機能を提供することで簡単に他の人とデータを共有しやすくしています。